### Trial 1 : RNN (LSTM) 
can handle sequential data (sentences) effectively
1. Embedding layer
2. LSTM layer (sequence processing)
3. Two dense layers for classification


In [2]:
# IMPORTS
import utils
from tensorflow import keras
import numpy as np

c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


#### 1. Load cleaned dataset

In [3]:
dataset_path = './final_intents.json'
unique_intents, corpus, corpus_intents, responses = utils.load_data(dataset_path)

# print shapes and sizes of the dataset
print('Number of unique intents:', len(unique_intents))
print('Number of responses:', len(responses))
print('Number of examples:', len(corpus))
print('Number of examples:', len(corpus_intents))

# print samples of the dataset
print('Unique intents:', unique_intents)
print('Responses:', responses)
print('Samples:', corpus[:5])

Number of unique intents: 14
Number of responses: 14
Number of examples: 340
Number of examples: 340
Unique intents: ['Varibale Declaration', 'Function Declaration', 'Class Declaration', 'Assignment Operation', 'Conditional Statement', 'Iterative Statement', 'Array Operation', 'Bitwise Operation', 'Mathematical Operation', 'Membership Operation', 'Casting', 'I/O Operation', 'Assertion', 'Libraries']
Responses: ['Variable declaration intent detected', 'function declaration intent detected', 'class declaration intent detected', 'assignment intent detected', 'conditional statement intent detected', 'iterative statement intent detected', 'array operation intent detected', 'bitwise operation intent detected', 'mathematical operation intent detected', 'membership operation intent detected', 'casting intent detected', 'I/O operation intent detected', 'assertion intent detected', 'library intent detected']
Samples: ['temp equals string', 'y equals', 'x equals', 'is finished equals false', 'is 

#### 2. Tokenizing and Embedding 

In [4]:
# create a tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')

# fit the tokenizer on the corpus -> updates internal vocabulary based on corpus
tokenizer.fit_on_texts(corpus)

# convert the corpus to sequences of integers -> each word is replaced by its index in the vocabulary for each sentence
sequences = tokenizer.texts_to_sequences(corpus)

# pad the sequences to the same length -> add padding tokens to the beginning of each sequence to fit the longest sequence
padded_sequences = keras.preprocessing.sequence.pad_sequences(sequences, padding='pre')

# get the number of unique words (vocabulary size)
vocab_size = len(tokenizer.word_index) + 1

# print the tokenizer properties
print('Vocabulary:', tokenizer.word_index)
print('Vocabulary Size:', vocab_size)
print('Shape of Input Sequence (# of examples, longest sequence length):', padded_sequences.shape)
print('Sample Input Sequence:', padded_sequences[0])

Vocabulary: {'<unk>': 1, 'a': 2, 'the': 3, 'to': 4, 'and': 5, 'is': 6, 'variable': 7, 'x': 8, 'named': 9, 'that': 10, 'name': 11, 'of': 12, 'y': 13, 'list': 14, 'it': 15, 'if': 16, 'function': 17, 'in': 18, 'create': 19, 'takes': 20, 'class': 21, 'assign': 22, 'define': 23, 'value': 24, 'new': 25, 'with': 26, 'set': 27, 'declare': 28, 'from': 29, 'string': 30, 'user': 31, 'array': 32, 'numbers': 33, 'check': 34, 'an': 35, 'equal': 36, 'than': 37, 'for': 38, 'b': 39, 'equals': 40, 'age': 41, 'values': 42, 'write': 43, 'not': 44, 'float': 45, 'message': 46, 'get': 47, 'loop': 48, 'while': 49, 'library': 50, 'false': 51, 'true': 52, 'import': 53, 'parameters': 54, 'num': 55, 'by': 56, 'type': 57, 'number': 58, 'valid': 59, 'bitwise': 60, 'their': 61, 'temp': 62, 'initiate': 63, 'returns': 64, 'print': 65, 'salary': 66, 'divided': 67, 'int': 68, 'var': 69, 'arr': 70, 'called': 71, 'two': 72, 'sum': 73, 'names': 74, 'cast': 75, 'input': 76, 'count': 77, 'add': 78, 'person': 79, 'put': 80, '

#### 3. Feature Extraction
map intents to indices

In [5]:
# dictionary that maps each intent to a unique index
intent_to_index = {intent: index for index, intent in enumerate(unique_intents)}

# list for each sentence mapped to its corresponding intent index 
corpus_intent_mapped_to_index = [intent_to_index[intent] for intent in corpus_intents]

# the number of classes to classify a sentence into
number_of_classes = len(intent_to_index)

# convert intent_to_index to index_to_intent 
index_to_intent = {index: intent for intent, index in intent_to_index.items()} 

print(index_to_intent)

# one hot encoding for the intents -> length of each vector is equal to the number of classes
# each sequence in the dataset is represented as a one-hot encoded vector that represents the intent of the sequence
one_hot_encoded_intents = keras.utils.to_categorical(corpus_intent_mapped_to_index, number_of_classes)

print('Categorial vector shape:', one_hot_encoded_intents.shape)

{0: 'Varibale Declaration', 1: 'Function Declaration', 2: 'Class Declaration', 3: 'Assignment Operation', 4: 'Conditional Statement', 5: 'Iterative Statement', 6: 'Array Operation', 7: 'Bitwise Operation', 8: 'Mathematical Operation', 9: 'Membership Operation', 10: 'Casting', 11: 'I/O Operation', 12: 'Assertion', 13: 'Libraries'}
Categorial vector shape: (340, 14)


#### 4. Building the Model

In [6]:
# the input is the the padded sequences with the target value being the one-hot encoded intents
input_dimenstion = len(unique_intents)

# the output is the one-hot encoded intents
output_dimenstion = one_hot_encoded_intents.shape[1]

print(f"Input Dimension: {input_dimenstion}, Output Dimension: {output_dimenstion}")

# Model description
# The model is a sequential model that consists of:
# 1. An embedding layer that converts the input sequences to dense vectors of fixed size
# 2. A Bidirectional LSTM layer that processes the input sequences in both directions
# 3. A Dense layer with 64 units and ReLU activation function
# 4. A Dropout layer with a dropout rate of 0.5
# 5. A Dense layer with the output dimension and softmax activation function for multi-class classification

# define parameters
epochs = 100

# the embedding dimension is the size of the vector for which each word is represented
# the embedding layer of a neural network, output_dim refers to the size of the dense vectors that the layer will generate for each input token (word). 
# essentially, it is the number of dimensions in which each word will be represented.
embedding_dimension = 128

# lstm units
lstm_units = 64

model = keras.Sequential([
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dimension),
    keras.layers.Bidirectional(keras.layers.LSTM(lstm_units, dropout=0.2)),
    keras.layers.Dense(lstm_units, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(output_dimenstion, activation='softmax')
])

optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


Input Dimension: 14, Output Dimension: 14



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         43648     
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 14)                910       
                                                                 
Total params: 151630 (592.30 KB)
Trainable params: 151630 (592.30 KB)
Non-trainable params: 0 (0.00 Byte)
________________

#### 5. Train the Model

In [7]:
model.fit(padded_sequences, one_hot_encoded_intents, epochs=epochs, verbose=0)

#### 6. Testing the Model

In [8]:
user_input = input("Sentence:")

# actual_intent = input("Intent:")

test_sequences = tokenizer.texts_to_sequences([user_input])

test_padded_sequences = keras.preprocessing.sequence.pad_sequences(test_sequences, padding='pre')

predictions = model.predict(test_padded_sequences)

predicted_intent_index = np.argmax(predictions)

predicted_intent = index_to_intent[predicted_intent_index]

print(f"Predicted Intent: {predicted_intent}")

# loss, accuracy = model.evaluate(test_padded_sequences, np.array([actual_intent]))

1/1 [==============================] - 1s 915ms/step
Predicted Intent: Varibale Declaration
